In [1]:
import numpy as np
import pandas as pd
from array import array
from PIL import Image
import os

In [2]:
labels = pd.read_csv("train_labels.csv")
metadata = pd.read_csv("train_metadata.csv")
array_2000 =[]

In [3]:
metadata.head()

,Image ID,Storm ID,Relative Time,Ocean
0,nhe_000,nhe,0,2
1,nhe_001,nhe,1800,2
2,nhe_002,nhe,3600,2
3,nhe_003,nhe,5402,2
4,nhe_004,nhe,9001,2


In [4]:
def channel3(img):
    empty_3d_array = np.empty((128, 128, 3))
    empty_3d_array[:,:,0]=np.array(img)
    empty_3d_array[:,:,1]=np.array(img)
    empty_3d_array[:,:,2]=np.array(img)
    return empty_3d_array
    

In [ ]:
def crop_center(image, new_width, new_height):
    # Get the current dimensions of the image
    width, height = image.size
    
    # Calculate the coordinates for the crop
    left = (width - new_width) // 2
    top = (height - new_height) // 2
    right = (width + new_width) // 2
    bottom = (height + new_height) // 2
    
    # Perform the crop
    cropped_image = image.crop((left, top, right, bottom))
    
    return cropped_image




In [5]:
def action_on_image_change(previous_id, current_id,df, labels_array, image_array):
    print(f"Name changed from {previous_id} to {current_id}")

    # Initialize previous name with None
    #previous_id = None
    # Initialize counter for the number of rows processed for the current ID
    rows_processed_for_current_id = 0
    start_index=0
    folder_path = "train/"
    
    # Loop over DataFrame rows
    for index, row in df.iterrows():
        current_id = row['Storm ID']

        # Check if the name has changed
        filename = row["Image ID"]+".jpg"
        file_path=os.path.join(folder_path, filename)
            
        if os.path.exists(file_path):
                #if rows_processed_for_current_id<10:
            
            try:
                img = Image.open(file_path)
                #resized_img = img.resize((128, 128))
                cropped_image=crop_center(img, 128, 128)
                image_array.append(channel3(cropped_image))
                
                img.close()
                labels_array.append(action_on_id_change(row["Image ID"] ))
            except:
                print("cannot identify image file",file_path)
                
        else:
            continue
        
    return labels_array, image_array
                
           

In [6]:
def switch_case(argument):
    return {
        1: [1,0,0,0,0],
        2: [0,1,0,0,0],
        3: [0,0,1,0,0],
        4: [0,0,0,1,0],
        5: [0,0,0,0,1]
    }.get(argument, "Invalid option")

In [7]:
def action_on_id_change(image_id):
    storm_array=[]
    name_to_find = image_id
    result_loc = labels.loc[labels['Image ID'] == name_to_find, 'Wind Speed'].values
    value_loc=result_loc[0]
    if ((value_loc>=15) & (value_loc<=45)):
        storm_array = switch_case(1)
    elif ((value_loc>45) & (value_loc<=80)):
        storm_array = switch_case(2) 
    elif (value_loc>80 & value_loc<=110):
        storm_array = switch_case(3)
    elif (value_loc>110 & value_loc<=150):
        storm_array = switch_case(4)
    elif (value_loc>150 & value_loc<=190):
        storm_array = switch_case(5)
    return storm_array
    

In [8]:
labels_array, image_array= action_on_image_change('abc', None, metadata, [],[])

Name changed from abc to None
cannot identify image file train/nhe_000.jpg
cannot identify image file train/nhe_001.jpg
cannot identify image file train/nhe_002.jpg
cannot identify image file train/nhe_003.jpg
cannot identify image file train/nhe_004.jpg
cannot identify image file train/nhe_005.jpg
cannot identify image file train/nhe_006.jpg
cannot identify image file train/nhe_007.jpg
cannot identify image file train/nhe_008.jpg
cannot identify image file train/nhe_009.jpg
cannot identify image file train/nhe_010.jpg
cannot identify image file train/nhe_011.jpg
cannot identify image file train/nhe_012.jpg
cannot identify image file train/nhe_013.jpg
cannot identify image file train/nhe_014.jpg
cannot identify image file train/epx_000.jpg
cannot identify image file train/epx_001.jpg
cannot identify image file train/epx_002.jpg
cannot identify image file train/epx_003.jpg
cannot identify image file train/epx_004.jpg
cannot identify image file train/epx_005.jpg
cannot identify image fil

cannot identify image file train/icx_000.jpg
cannot identify image file train/icx_001.jpg
cannot identify image file train/icx_002.jpg
cannot identify image file train/icx_003.jpg
cannot identify image file train/icx_004.jpg
cannot identify image file train/wlz_000.jpg
cannot identify image file train/wlz_001.jpg
cannot identify image file train/wlz_002.jpg
cannot identify image file train/wlz_003.jpg
cannot identify image file train/wlz_004.jpg
cannot identify image file train/wlz_005.jpg
cannot identify image file train/wlz_006.jpg
cannot identify image file train/wlz_007.jpg
cannot identify image file train/wlz_008.jpg
cannot identify image file train/wlz_009.jpg
cannot identify image file train/wlz_010.jpg
cannot identify image file train/wlz_011.jpg
cannot identify image file train/wlz_012.jpg
cannot identify image file train/wlz_013.jpg
cannot identify image file train/wlz_014.jpg
cannot identify image file train/wlz_015.jpg
cannot identify image file train/wlz_016.jpg
cannot ide

In [9]:
file_path_label = "wind_label_366X366.npy"
file_path = "wind_366X366.npy"
np.save(file_path, image_array)
np.save(file_path_label, labels_array)